In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `d:\Gits\YardSale.jl\benchmark`


In [2]:
using YardSale
using BenchmarkTools
using Graphs

# First benchmark: ODEs

In [3]:
# TODO

# Second benchmark : EYSM base

In [4]:
# TODO

# Third benchmark: every interaction and taxation mode over a single network

In [5]:
# Instantiate a benchmark group
suite = BenchmarkGroup()

# Graph parameters
N = 128
k = 8
p = k/(N-1)
g = erdos_renyi(N, p, seed=42)
# Simulation parameters
W_N = 1.0f0
chi = 1.0f0
zeta = 1.0f0
f = 0.01
steps = 10000 * N
seed = 42


interaction_modes = ["A", "B"]
taxation_modes = ["A", "B"]

# Add benchmarks
suite["IM_TM"] = BenchmarkGroup()

# Add some benchmarks to the "IM_TM" group
suite["IM_TM"]["A,A"] = @benchmarkable EYSM_net_full($g, $W_N, "A", "A", $chi, $zeta, $f, $steps, $seed)
suite["IM_TM"]["A,B"] = @benchmarkable EYSM_net_full($g, $W_N, "A", "B", $chi, $zeta, $f, $steps, $seed)
suite["IM_TM"]["B,A"] = @benchmarkable EYSM_net_full($g, $W_N, "B", "A", $chi, $zeta, $f, $steps, $seed)
suite["IM_TM"]["B,B"] = @benchmarkable EYSM_net_full($g, $W_N, "B", "B", $chi, $zeta, $f, $steps, $seed)
# Execute `tune!` to find the optimal sample configuration for each benchmark
tune!(suite)

1-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "IM_TM" => 4-element BenchmarkTools.BenchmarkGroup:
	  tags: []
	  "B,A" => Benchmark(evals=1, seconds=5.0, samples=10000)
	  "B,B" => Benchmark(evals=1, seconds=5.0, samples=10000)
	  "A,B" => Benchmark(evals=1, seconds=5.0, samples=10000)
	  "A,A" => Benchmark(evals=1, seconds=5.0, samples=10000)

In [11]:
t = run(suite, verbose=true)

(1/1) benchmarking "IM_TM"...
  (1/4) benchmarking "B,A"...
  done (took 18.5413815 seconds)
  (2/4) benchmarking "B,B"...
  done (took 36.9523583 seconds)
  (3/4) benchmarking "A,B"...
  done (took 52.8755378 seconds)
  (4/4) benchmarking "A,A"...
  done (took 20.2119828 seconds)
done (took 130.4102364 seconds)


1-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "IM_TM" => 4-element BenchmarkTools.BenchmarkGroup:
	  tags: []
	  "B,A" => Trial(8.576 s)
	  "B,B" => Trial(17.723 s)
	  "A,B" => Trial(26.083 s)
	  "A,A" => Trial(9.179 s)

In [24]:
t["IM_TM"]["A,A"]

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 9.179 s (0.00% GC) to evaluate,
 with a memory estimate of 5.97 MiB, over 30025 allocations.

In [49]:
# Plot the results
im_tm = ["A,A", "A,B", "B,A", "B,B"]
using Plots
p = plot(palette=:julia, yminorticks=true,tickfontsize=14,labelfontsize=15,framestyle=:box,
legendfontsize=14,size=(640,480))
for (i,imtm) in enumerate(im_tm)
    p = scatter!(p,[i],[t["IM_TM"][imtm].times[1]/1e9], label=imtm, ms=6)
end

xticks!(1:4, im_tm)
ylabel!("Time (s)")
xlabel!("Interaction mode, Taxation mode")

savefig(p, "benchmark_IM_TM_firstversion.png")

"d:\\Gits\\YardSale.jl\\benchmark\\benchmark_IM_TM_firstversion.png"